In [38]:
from warnings import filterwarnings
filterwarnings("ignore")

In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import spacy
import re
import fasttext

In [24]:
data = pd.read_csv("ecommerce_dataset.csv",sep=',',header=None,names=["category","description"])
data.head()

,category,description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50425 entries, 0 to 50424
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   category     50425 non-null  object
 1   description  50424 non-null  object
dtypes: object(2)
memory usage: 788.0+ KB


In [26]:
data.describe()

,category,description
count,50425,50424
unique,4,27802
top,Household,Think & Grow Rich About the Author NAPOLEON HI...
freq,19313,30


In [27]:
data.count()

category       50425
description    50424
dtype: int64

In [28]:
data.dtypes

category       object
description    object
dtype: object

In [29]:
data.index

RangeIndex(start=0, stop=50425, step=1)

In [30]:
data.columns

Index(['category', 'description'], dtype='object')

In [31]:
data.isnull().sum()

category       0
description    1
dtype: int64

In [39]:
df = data.copy()

In [40]:
df.dropna(axis=0,how='any',inplace=True)

In [41]:
df.category.value_counts()

category
Household                 19313
Books                     11820
Electronics               10621
Clothing & Accessories     8670
Name: count, dtype: int64

In [42]:
df["category"].replace(to_replace="Clothing & Accessories",value="Clothing_Accessories",inplace=True)
df.shape

(50424, 2)

In [43]:
df.category.unique()

array(['Household', 'Books', 'Clothing_Accessories', 'Electronics'],
      dtype=object)

In [44]:
df["category"] = "__label__" + df["category"]

In [57]:
df["category_description"] = df["category"] + " " + df["description"]

In [58]:
df.head()

,category,description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__Household SAF Flower Print Framed Pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__Household Incredible Gifts India Wood...


In [59]:
df["category_description"][0]

'__label__Household Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and so

In [60]:
def preprocess(text):
    text = re.sub("[^\w\s\']"," ",text)
    text = re.sub(' +',' ',text)
    text = text.strip().lower()
    return text

In [61]:
df["category_description"] = df["category_description"].apply(preprocess)
df["category_description"][0]

'__label__household paper plane design framed wall hanging motivational office decor art prints 8 7 x 8 7 inch set of 4 painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it this is an special series of paintings which makes your wall very beautiful and gives a royal touch this painting is ready to hang you would be proud to possess this unique painting that is a niche apart we use only the most modern and efficient printing technology on our prints with only the and inks and precision epson roland and hp printers this innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime we print solely with top notch 100 inks to achieve brilliant and true colours due to their high level of uv resistance our prints retain their beautiful colours for many years add colour and style to your living space with this digitally printed painting some are for pleasure and some for eternal bli

In [62]:
train,test = train_test_split(df,test_size=0.2)

In [63]:
train.shape

(40339, 3)

In [64]:
test.shape

(10085, 3)

In [65]:
train.to_csv("ecommerce.train",columns=["category_description"],index=False,header=False)
test.to_csv("ecommerce.test",columns=["category_description"],index=False,header=False)

In [67]:
model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

(10084, 0.9686632288774296, 0.9686632288774296)

In [68]:
x = model.test("ecommerce.test")

In [70]:
print(f"Number of test samples: {x[0]}\nPrecision: {x[1]}\nRecall: {x[2]}")

Number of test samples: 10084
Precision: 0.9686632288774296
Recall: 0.9686632288774296


In [71]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99510348]))

In [72]:
model.predict("ockey men's cotton t shirt fabric details 80 cotton 20 polyester super combed cotton rich fabric")

(('__label__clothing_accessories',), array([1.00001001]))

In [73]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000978]))

In [74]:
model.get_nearest_neighbors("painting")

[(0.9987078309059143, 'polyguards'),
 (0.9987078309059143, 'distributer'),
 (0.9986568093299866, "'dough"),
 (0.998652458190918, "'live"),
 (0.9986458420753479, 'muticolour'),
 (0.998638391494751, '96pcs'),
 (0.9986348152160645, 'coleslaw'),
 (0.9986153244972229, 'tivoli'),
 (0.9986096620559692, 'tat3a011'),
 (0.9985870122909546, 'intensify')]

In [75]:
model.get_nearest_neighbors("sony")

[(0.9991397261619568, 'aluminumsize'),
 (0.9991397261619568, '9cmpackage'),
 (0.9990490674972534, 'w90'),
 (0.9990490674972534, 'd335'),
 (0.9990490674972534, 'mms8085b'),
 (0.9990490674972534, 'h165'),
 (0.9990490674972534, 'd78'),
 (0.9990490674972534, 'h312'),
 (0.9990490674972534, 'w170'),
 (0.9990209937095642, 'manga')]